In [15]:
# Setup and Imports
%load_ext autoreload
%autoreload 2

import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from uncertainties import ufloat
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# Add the src directory to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import parse_sample_pits, pit_to_slab, pit_to_slab_above_weak_layer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')

Successfully parsed 50278 files
Failed to parse 0 files


In [17]:

# Convert pits to mech params pit objects
all_slabs = []

for pit in all_pits:
    slab = pit_to_slab(pit)
    all_slabs.append(slab)


In [18]:
print(len(all_slabs))

50278


In [19]:
# Calculate static load from slabs


# Create Slabs Above Weak Layers

Create slabs for each weak layer definition:
- **layer_of_concern**: Layers above the indicated layer of concern
- **CT_failure_layer**: Layers above CT failure layer (Q1/SC/SP fracture character)
- **ECTP_failure_layer**: Layers above ECTP failure layer (with propagation)


In [20]:
# Create slabs above layer of concern
slabs_above_loc = []

for pit in all_pits:
    slab = pit_to_slab_above_weak_layer(pit, "layer_of_concern")
    if slab:
        slabs_above_loc.append({
            'pit_id': pit.core_info.pit_id,
            'slab': slab,
            'num_layers': len(slab.layers),
            'total_thickness': slab.total_thickness,
            'angle': slab.angle
        })

print(f"Created {len(slabs_above_loc)} slabs above layer of concern")


Created 34690 slabs above layer of concern


In [21]:
# Create slabs above CT failure layer
slabs_above_ct = []

for pit in all_pits:
    slab = pit_to_slab_above_weak_layer(pit, "CT_failure_layer")
    if slab:
        slabs_above_ct.append({
            'pit_id': pit.core_info.pit_id,
            'slab': slab,
            'num_layers': len(slab.layers),
            'total_thickness': slab.total_thickness,
            'angle': slab.angle
        })

print(f"Created {len(slabs_above_ct)} slabs above CT failure layer")


Created 13633 slabs above CT failure layer


In [22]:
# Create slabs above ECTP failure layer
slabs_above_ectp = []

for pit in all_pits:
    slab = pit_to_slab_above_weak_layer(pit, "ECTP_failure_layer")
    if slab:
        slabs_above_ectp.append({
            'pit_id': pit.core_info.pit_id,
            'slab': slab,
            'num_layers': len(slab.layers),
            'total_thickness': slab.total_thickness,
            'angle': slab.angle
        })

print(f"Created {len(slabs_above_ectp)} slabs above ECTP failure layer")


Created 12683 slabs above ECTP failure layer


In [23]:
# Summary statistics for slabs above weak layers
print("="*80)
print("SLAB STATISTICS BY WEAK LAYER DEFINITION")
print("="*80)

# Create dataframes for each definition
df_loc = pd.DataFrame(slabs_above_loc)
df_ct = pd.DataFrame(slabs_above_ct)
df_ectp = pd.DataFrame(slabs_above_ectp)

# Summary statistics
summary_data = []
for name, df in [("Layer of Concern", df_loc), ("CT Failure", df_ct), ("ECTP Failure", df_ectp)]:
    if len(df) > 0:
        summary_data.append({
            'Weak Layer Definition': name,
            'Total Slabs': len(df),
            'Avg Layers': df['num_layers'].mean(),
            'Avg Thickness (cm)': df['total_thickness'].mean(),
            'Avg Angle (deg)': df['angle'].mean()
        })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))
print("="*80)


SLAB STATISTICS BY WEAK LAYER DEFINITION
Weak Layer Definition  Total Slabs  Avg Layers  Avg Thickness (cm)  Avg Angle (deg)
     Layer of Concern        34690    3.765264           51.671750        23.590948
           CT Failure        13633    3.827404           52.429029        23.579990
         ECTP Failure        12683    4.136797           53.573628        24.531026
